In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#loaderの設定などの練習

import json

In [3]:
from src.CommonCrawlDataset import CommonCrawlDataset

In [4]:
cc_loader=CommonCrawlDataset()

In [5]:
it=iter(cc_loader)

In [9]:
next(it)

{'text': '現在会員でない方も、2月5日(月)までにご入会手続きを完了された方につきましては、会員枠にて抽選無しでご参加いただけます。'}

In [13]:
import glob
import random
jsonl_path_list=glob.glob("../data/dedup_categorized/*/*.jsonl")
random.shuffle(jsonl_path_list)
cid_to_path_list={}
for jsonl_path in jsonl_path_list:
    cid=jsonl_path.split("/")[-2]
    if cid not in cid_to_path_list:
        cid_to_path_list[cid]=[]
    cid_to_path_list[cid].append(jsonl_path)

In [14]:
max_texts=10
cid_to_texts={}
for cid, path_list in cid_to_path_list.items():
    texts=[]
    for path in path_list:
        with open(path) as f:
            for i, line in enumerate(f):
                if i>=max_texts:
                    break
                texts.append(json.loads(line)["text"])
    cid_to_texts[cid]=texts

In [23]:
import sys
sys.path.append("../01web_codes")
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec, texts2classes

t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [25]:
import numpy as np
cid_to_vecs={}
check_length=100
for cid, texts in tqdm(cid_to_texts.items()):
    vecs = [t2v.text2vec(i[:check_length]) for i in texts]
    vecs=np.array(vecs)
    vecs=np.mean(vecs, axis=0)
    cid_to_vecs[cid]=vecs



  0%|          | 0/791 [00:00<?, ?it/s]

100%|██████████| 791/791 [00:25<00:00, 31.27it/s] 


In [26]:
n_clusters=5
from sklearn.cluster import MiniBatchKMeans,KMeans
vec_array=np.array(list(cid_to_vecs.values()))
# k-meansクラスタリング
print("clustering...")
kmeans = KMeans(n_clusters=n_clusters, random_state=1).fit(vec_array)

clustering...


In [49]:
cid_to_label={}
for cid, vec in cid_to_vecs.items():
    label=kmeans.predict([vec])[0]
    cid_to_label[cid]=label

In [54]:
label_to_path_list={}
for cid, label in cid_to_label.items():
    if str(label) not in label_to_path_list:
        label_to_path_list[str(label)]=[]
    path=f"../data/dedup_categorized/{cid}/*.jsonl"
    label_to_path_list[str(label)].append(path)


In [55]:
import json
with open("../data/clustered.json", "w") as f:
    json.dump(label_to_path_list, f, indent=4)

In [102]:
#行数をwcで数える
import os
import subprocess
label_to_article_count={}
for label, s_path_list in label_to_path_list.items():
    label_to_article_count[label]=0
    for s_path in s_path_list:
        for path in glob.glob(s_path):
            res=os.system(f"wc -l {path}")
            result = subprocess.run(["wc", "-l"] + path.split(), capture_output=True, text=True)
            # 出力から合計行数を抽出
            count=result.stdout.split(" ")[0]
            label_to_article_count[label] += int(count)

243 ../data/dedup_categorized/17/cc100_ja_120l.jsonl
228 ../data/dedup_categorized/17/cc100_ja_124l.jsonl
43 ../data/dedup_categorized/17/oscar_ja_132l.jsonl
25 ../data/dedup_categorized/17/oscar_ja_394l.jsonl
30 ../data/dedup_categorized/17/c4-ja.tfrecord-00382-of-01024.jsonl
29 ../data/dedup_categorized/17/c4-ja.tfrecord-00213-of-01024.jsonl
24 ../data/dedup_categorized/17/c4-ja.tfrecord-00912-of-01024.jsonl
54 ../data/dedup_categorized/17/cc2021_batch8-7_0l.jsonl
58 ../data/dedup_categorized/17/cc2021_batch5-6_0l.jsonl
33 ../data/dedup_categorized/17/oscar_ja_46l.jsonl
19 ../data/dedup_categorized/17/c4-ja.tfrecord-00751-of-01024.jsonl
27 ../data/dedup_categorized/17/c4-ja.tfrecord-00301-of-01024.jsonl
23 ../data/dedup_categorized/17/c4-ja.tfrecord-00410-of-01024.jsonl
48 ../data/dedup_categorized/17/cc2021_batch27-1_0l.jsonl
32 ../data/dedup_categorized/17/c4-ja.tfrecord-00930-of-01024.jsonl
2 ../data/dedup_categorized/17/9l.jsonl
28 ../data/dedup_categorized/17/c4-ja.tfrecord-0038

In [104]:

with open("../data/clustered_n.json", "w") as f:
    json.dump(label_to_article_count, f, indent=4)

In [200]:
cc1_loader=CommonCrawlDataset(label_to_path_list["1"])
cc1_iter=iter(cc1_loader)
for i in range(5):
    print(next(cc1_iter))

{'text': '地方創生スタートから2年成否を分けるカギはどこに?\n地方創生の検証と対策\n「地方創生」が政策として掲げられて早2年。各地の状況をつぶさに観察すると、停滞ないしは衰退し続けるところと、上向くところの「差異」が徐々に顕在化してきている。それは何に起因するのか?創生の成否を分けるカギについて、具体的に検討してみたい。\n創生成功のカギ「長期的視点」\n現政権が推進する「地方創生」は、国として大きな枠組みを用意し、必要に応じてサポートを行うものの、基本的には、各地域が、自らの叡智によって、その地でサバイブすることを求める点に特徴がある。\nここで言う「叡智」は2つある。\n1つ目は、「長期的視点」である。「国家100年の計」というが、地域の創生も本来それに準じるものだ。50年後、100年後にどんな地域になっていたいのかが“明確"で、地域内外の人々が、「自分もその一助となりたい」と思えるほど“魅力的"な、そして“環境変化の風雪に耐え得る力のある"理念・哲学をもった長期的プランニングが望まれる。\nザルツブルク音楽祭を核に、音楽・文化の街として世界中から観光客が訪れるオーストリアのザルツブルク。約100年前は、何の特徴もない寂れ果てた街だったPhoto by Wikimedia\n一例を挙げよう。オーストリアのザルツブルクは、“名実ともに世界一"と評される祝祭(＝ザルツブルク音楽祭)を核に、多数の自然・歴史遺産を擁する人気観光リゾートである。しかし、100年近く前、同市は、人口3万の過疎の町であった。第一次大戦に破れた祖国オーストリア(ハプスブルク帝国)は、欧州の周縁部の1小国になり下がり、ザルツブルクも、“モーツァルト生誕地"という以外、何の特徴もない寂れ果てた街だったのである。\nところが、ここに、「演劇界の皇帝」マックス・ラインハルト、文豪フーゴー・フォン・ホーフマンスタール、大作曲家リヒャルト・シュトラウスが集い、“モーツァルトの音楽と演劇理念の復権・再生"を目指し、1920年夏、「ザルブルク・フェスティバル」(ザルツブルク音楽祭)を創始する。\nそして、その理念の延長上に、やがて、冬の「モーツァルト週間」(1956～)、春の「復活祭音楽祭」(「指揮界の帝王」ヘルベルト・フォン・カラヤンが創始、1967～)などの音楽祭群を加えてゆく。さらには、ミ